In [40]:
import autograd.numpy as np
from autograd import grad, hessian
import sys

In [97]:
def subproblem_solver_cauchyPoint(g, B, delta):
    gT_b_g = np.dot(np.dot(g, B), g)
    gT_g = np.dot(g, g)
    g_norm = np.linalg.norm(g)

    if gT_b_g > 0 and abs(gT_g / gT_b_g) * g_norm < delta:
        alpha = gT_g / gT_b_g
    else:
        alpha = delta / g_norm

    return -alpha * g

In [98]:
def sr1_update(B, s, y):
    """
    Update the Hessian approximation B using the SR1 formula
    """
    u = y - B.dot(s)
    denom = u.dot(s)
    if abs(denom) > 1e-8:
        B = B + np.outer(u,u) / denom;
    return B

In [99]:
def newton_with_sr1(x0, f, g, B, delta, tol=1e-6, max_iter=10000):
    """
    Newton method with SR1 update
    """
    x = x0;
    for i in range(max_iter):
        # solve the subproblem
        s = subproblem_solver_cauchyPoint(g(x), B, delta);
        # update x
        x = x + s;
        # update B
        B = sr1_update(B, s, g(x) - g(x - s))
        # check the stopping criterion
        if np.linalg.norm(s) < tol:
            break
    return x;

In [100]:
f = lambda x: 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2
g = grad(f)
B = hessian(f)
x0 = np.array([0., 0.])
delta = 1
x = newton_with_sr1(x0, f, g, B(x0), delta);
print(x)
print(f(x))

[0.99957629 0.99915159]
1.7966730196262525e-07
